# Time Series & Historical Query Analysis with Python

Welcome to this tutorial where we'll demonstrate how to work with large datasets in KDB-X to analyze time-series data using KDB-X Python.

One of the key features of this client is its ability to handle huge volumes of data with exceptional speed and efficiency. Whether it's reading massive datasets, performing time-based aggregations, or joining data from different sources, it excels at time-series analysis through its tight connection to KDB-X.

By the end of this tutorial, you'll have a clear understanding of how to create, manipulate, store, and analyze data using KDB-X Python. Along the way, we'll introduce several key concepts that are fundamental to working with data in KDB-X.


Here, we'll cover:
- Creating a large time-series dataset from scratch
- Saving this data to a database on disk
- Scaling database to 300 Million rows
- Performing time-based aggregations to analyze trends over time
- Using asof joins (aj) to combine time-series data (e.g., matching trades to quotes)

## 1. Prerequisites

Before starting this tutorial, ensure that you have KDB-X along with KDB-X Python installed on your system. If you haven't installed these yet, you can sign up at https://kdb-x.kx.com/sign-in.

## 2. Create the Time Series Dataset

Let’s start by creating a sample dataset to work with. This dataset will simulate trade data over a period of time, with random values for price, size, and symbols.

Firstly you can import KDB-X and a number of additional libraries into your Python session:

In [ ]:
import pykx as kx
from datetime import date
from datetime import timedelta
import numpy as np
import os
import psutil
from pathlib import Path

Next download the [stocks.txt](https://github.com/KxSystems/tutorials/blob/main/KDB-X/Python/stocks.txt) file locally and make sure the path below points to it.

In [2]:
with open('stocks.txt', 'r') as f:
    py_symbols = f.read().split('\n')

In [3]:
syms = kx.random.random(100, py_symbols)

Above we are selecting 100 random stock symbols from an external text file containing a list of real stock tickers.

Now we can generate a time-series dataset:

In [4]:
n = 20000000;
day = date(2025, 1, 1);
trade = kx.Table(data = {
     'time': day + kx.random.random(n, kx.q('24:00:00')),
     'sym': kx.random.random(n, syms),
     'price': kx.random.random(n, 100.0),
     'size': kx.random.random(n, 1000)
    }
).sort_values('time')

Here's a breakdown of what's happening:
- `n = 2000000` sets the number of rows we want to generate
- We define a new table using `kx.Table` with syntax similar to that used by Pandas
- The function `kx.random.random` is then used to create `n` randomly generated
    - `time` is populated with timestamps starting from midnight and increasing across a 24-hour period, with a random offset to simulate a spread of trades. These are then sorted in ascending order using `kx.q.asc`.
    - `sym` is populated with random symbols, selected from a list.
    - `price` and trade `size` are randomly generated

This table is now available in memory to investigate and query. Let's take a quick look at the following:

- Row count using `len`
- Schema details using the attributes `dtypes`
- Retrieve the first 10 rows using the method `head`

In [5]:
len(trade)

20000000

In [6]:
trade.dtypes

,columns,datatypes
,,
0,time,"""kx.TimestampAtom"""
1,sym,"""kx.SymbolAtom"""
2,price,"""kx.FloatAtom"""
3,size,"""kx.LongAtom"""


In [7]:
trade.head(10)

,time,sym,price,size
,,,,
0,2025.01.01D00:00:00.000000000,SAVA,86.12543,644
1,2025.01.01D00:00:00.000000000,JAMF,16.51214,828
2,2025.01.01D00:00:00.000000000,TBBK,10.68305,327
3,2025.01.01D00:00:00.000000000,WVVIP,89.58529,990
4,2025.01.01D00:00:00.000000000,LGCL,38.29104,904
5,2025.01.01D00:00:00.000000000,MNST,40.16916,264
6,2025.01.01D00:00:00.000000000,DSWL,70.14543,633
7,2025.01.01D00:00:00.000000000,APCX,32.23879,629
8,2025.01.01D00:00:00.000000000,KPLT,18.70299,276



### SQL Support 
SQL querying is supported in the latest version of KDB-X, making it easier to work with familiar SQL syntax directly. SQL can be invoked by running:
```python
kx.q.sql("SELECT * FROM $1 LIMIT 10", trade)
```
For more information see [SQL documentation](https://code.kx.com/insights/core/sql.html#running-sql).

In [8]:
kx.q.sql("SELECT * FROM $1 LIMIT 10", trade)

,time,sym,price,size
,,,,
0,2025.01.01D00:00:00.000000000,SAVA,86.12543,644
1,2025.01.01D00:00:00.000000000,JAMF,16.51214,828
2,2025.01.01D00:00:00.000000000,TBBK,10.68305,327
3,2025.01.01D00:00:00.000000000,WVVIP,89.58529,990
4,2025.01.01D00:00:00.000000000,LGCL,38.29104,904
5,2025.01.01D00:00:00.000000000,MNST,40.16916,264
6,2025.01.01D00:00:00.000000000,DSWL,70.14543,633
7,2025.01.01D00:00:00.000000000,APCX,32.23879,629
8,2025.01.01D00:00:00.000000000,KPLT,18.70299,276


## 3.  Save Data to Disk

Once the data is generated, you’ll likely want to save it to disk for persistent storage.

Because we want the ability to scale, partitioning by date will be a good approach for this dataset. Without partitioning, queries that span large time periods would require scanning entire datasets, which can be very slow and resource-intensive. By partitioning data, KDB-X can limit the query scope to the relevant partitions, significantly speeding up the process.

First let's define some filepaths to make things easier to manage, you can change your `homeDir` to wherever you wish to save your data.

In [9]:
home_dir = Path.home()
db_dir = home_dir / 'data'

Now that we have access to the location you can initialize a `kx.DB` class object which you can use to interact with your database.

In [10]:
database = kx.DB(path = db_dir)

Now that you have the location for your database is available we can persist a table to our database

In [11]:
database.create(trade, 'trade', day, by_field = 'sym')

Writing Database Partition 2025-01-01 to table trade


Once persisted you can list the tables associated with the database.

In [12]:
database.tables

['trade']

Accessing the table for query via Python can be done by accessing the named table attribute, for example as follows accessing the `trade` table

In [13]:
database.trade

,date,sym,time,price,size
,,,,,
0,2025.01.01,ACOG,2025.01.01D00:00:00.000000000,90.01866,930
1,2025.01.01,ACOG,2025.01.01D00:00:00.000000000,2.025345,688
2,2025.01.01,ACOG,2025.01.01D00:00:01.000000000,24.0999,429
3,2025.01.01,ACOG,2025.01.01D00:00:02.000000000,5.307969,706
4,2025.01.01,ACOG,2025.01.01D00:00:02.000000000,38.31707,330
5,2025.01.01,ACOG,2025.01.01D00:00:03.000000000,93.01008,74
6,2025.01.01,ACOG,2025.01.01D00:00:03.000000000,77.77348,454
7,2025.01.01,ACOG,2025.01.01D00:00:04.000000000,78.41395,130
8,2025.01.01,ACOG,2025.01.01D00:00:05.000000000,5.631432,406


In [ ]:
psutil.disk_usage('.')

> **Note on Disk Space**: To create 300 Million rows this will require ~10GB of data space. If you have less than 10GB available, please reduce the days below otherwise you will run out of space.

Let's generate a list of new dates and copy the existing partition (2025.01.01) to these new dates. This may take ~60 seconds to complete.

In [ ]:
for i in range(1, 15):
   offset = timedelta(days=i)
   data = trade.update(kx.Column('time') + offset)
   database.create(data, 'trade', day + offset, by_field = 'sym')

Writing Database Partition 2025.01.02 to table trade
Writing Database Partition 2025.01.03 to table trade
Writing Database Partition 2025.01.04 to table trade
Writing Database Partition 2025.01.05 to table trade
Writing Database Partition 2025.01.06 to table trade
Writing Database Partition 2025.01.07 to table trade
Writing Database Partition 2025.01.08 to table trade
Writing Database Partition 2025.01.09 to table trade
Writing Database Partition 2025.01.10 to table trade
Writing Database Partition 2025.01.11 to table trade
Writing Database Partition 2025.01.12 to table trade
Writing Database Partition 2025.01.13 to table trade
Writing Database Partition 2025.01.14 to table trade
Writing Database Partition 2025.01.15 to table trade


In the above:
- Looping over 15 days and creating an updated date for each partition achieved by adding one day at each interval and retrieving the `date`.
- Using the `update` method to increase the date for the `time` columns used within the database where the `time` column has been retrieved using the `exec` method.
- Add the database partitions by running the `create` method on the previously initialised `database` object.


Once the partitions are created, we verify how much disk space was consumed and check the new partitions exist.

In [ ]:
print(psutil.disk_usage('.'))
for item in db_dir.iterdir():print(item)

To validate the amount of data within our loaded database we can run a `select` query on the data

In [15]:
database.trade.select(
    columns = kx.Column('time').count(),
    by = kx.Column('date')
)

,time
date,
2025.01.01,20000000
2025.01.02,20000000
2025.01.03,20000000
2025.01.04,20000000
2025.01.05,20000000
2025.01.06,20000000
2025.01.07,20000000
2025.01.08,20000000
2025.01.09,20000000


## 5. Time Series Analytics

Now that we have 300 Million rows of data, let's dive into some basic time-series analytics.

### Total Trade Volume Every Hour

Let's find a symbol to analyse from the randomly generated list we created earlier and then run our query. In the below query we're calculating the total traded volume size per-hour for our specified symbol across our 300 Million datapoints.

In [16]:
symbol = syms[0]
database.trade.select(
    columns = kx.Column('size').sum(),
    by = kx.Column('date') & kx.Column('time').minute.xbar(60),
    where = kx.Column('sym') == symbol
)

pykx.KeyedTable(pykx.q('
date       time | size   
----------------| -------
2025.01.01 00:00| 4080324
2025.01.01 01:00| 4214637
2025.01.01 02:00| 4168770
2025.01.01 03:00| 4284067
2025.01.01 04:00| 4198041
2025.01.01 05:00| 4066934
2025.01.01 06:00| 4261078
2025.01.01 07:00| 4198343
2025.01.01 08:00| 4208092
2025.01.01 09:00| 4170412
2025.01.01 10:00| 4169302
2025.01.01 11:00| 4148487
2025.01.01 12:00| 4152190
2025.01.01 13:00| 4308098
2025.01.01 14:00| 4108724
2025.01.01 15:00| 4191738
2025.01.01 16:00| 4227911
2025.01.01 17:00| 4165902
2025.01.01 18:00| 3974367
2025.01.01 19:00| 4194539
..
'))

#### Python querying & Temporal Arithmetic

In the following section we will make use of KDB-X Python commands for querying databases. This is outlined in more detail <a href="https://code.kx.com/pykx/3.1/user-guide/fundamentals/query/pyquery.html" target="_blank">here</a>.

KDB-X supports several temporal types and arithmetic between them. See here for a summary of <a href="https://code.kx.com/q/ref/#datatypes" target="_blank">datatypes</a>.

In this tutorial:

- The `time` column in the data has a type of timestamp, which includes both date and time values.
- We convert the `time` values to their minute values (including hours and minutes)
- We then aggregate further on time by using <a href="https://code.kx.com/pykx/3.1/api/columns.html#pykx.wrappers.Column.xbar" target="_blank">xbar</a> to bucket the minutes into hours (60-unit buckets)

### Weighted Average Price and Last Trade Price Every 15 Minutes

In [17]:
database.trade.select(
    columns = kx.Column('price').last().name('lastPx') &
              kx.Column('size').wavg(kx.Column('price')).name('vwapPx'),
    by = kx.Column('date') & kx.Column('time').minute.xbar(15),
    where = kx.Column('sym') == symbol
)

pykx.KeyedTable(pykx.q('
date       time | lastPx   vwapPx  
----------------| -----------------
2025.01.01 00:00| 94.40285 49.67349
2025.01.01 00:15| 18.49636 50.62056
2025.01.01 00:30| 42.26113 49.01375
2025.01.01 00:45| 42.58579 50.39329
2025.01.01 01:00| 5.000717 49.02052
2025.01.01 01:15| 42.85435 49.18247
2025.01.01 01:30| 91.1341  49.64506
2025.01.01 01:45| 58.96302 50.00214
2025.01.01 02:00| 23.91393 49.37068
2025.01.01 02:15| 10.80449 50.05471
2025.01.01 02:30| 68.10279 50.29423
2025.01.01 02:45| 62.52744 50.77956
2025.01.01 03:00| 43.55532 50.02526
2025.01.01 03:15| 19.10655 49.8321 
2025.01.01 03:30| 28.78066 49.86237
2025.01.01 03:45| 89.05311 51.3927 
2025.01.01 04:00| 4.520687 50.69658
2025.01.01 04:15| 90.6012  49.26168
2025.01.01 04:30| 39.96687 50.67064
2025.01.01 04:45| 64.76414 51.04639
..
'))

This is similar to the previous analytic, but this time we make use of the built in `wavg` function to find out the weighted average over time intervals.

In finance, volume-weighted averages give a more accurate reflection of a stock’s price movement by incorporating trading volume at different price levels. This can be especially useful in understanding whether a price move is supported by strong market participation or is just a result of a few trades.

The query processed 300 Million records in seconds, efficiently aggregating last price (`lastPx`) and volume-weighted-average price (`vwapPx`) for these trades. The use of `by date, 15 xbar time.minute` optimized the grouping, making the computation fast. This demonstrates the power of KDB-X/q for high-speed time-series analytics.

 ### SQL Comparison

A SQL version of this query above would look something like:

```sql
SELECT
    (array_agg(price ORDER BY time DESC))[1] AS lastPx,
    SUM(price * size) / NULLIF(SUM(size), 0) AS vwapPx,
    DATE_TRUNC('day', time),
    TRUNC(time, 'MI') + (FLOOR(TO_NUMBER(TO_CHAR(time, 'MI')) / 15) * INTERVAL '15' MINUTE)
FROM
    trade
WHERE
    sym = 'MSFT'
GROUP BY
    DATE_TRUNC('day', time),
    TRUNC(time, 'MI') + (FLOOR(TO_NUMBER(TO_CHAR(time, 'MI')) / 15) * INTERVAL '15' MINUTE)
ORDER BY
    DATE_TRUNC('day', time),
    TRUNC(time, 'MI') + (FLOOR(TO_NUMBER(TO_CHAR(time, 'MI')) / 15) * INTERVAL '15' MINUTE);
```

SQL is more complex due to several factors:
- **Time-series Calculations**: The SQL version involves the creation of custom logic for common time-series calculations such as volume-weighted-averages. In the q-sql version, these functionalities are implicit, and the syntax is more concise when working with vectors. The SQL equivalent requires custom definitions and is often more verbose leaving room for error.
- **Grouping and Aggregation**: In the q-sql version, grouping by date and a 15 minute window is done with a single, simple syntax, which is an efficient and intuitive way to express time bucketing. In SQL, similar behavior requires explicitly defining how time intervals are handled and aggregating the results using GROUP BY with custom time expressions which are often repeated throughout the query.
- **Temporal Formatting**: SQL queries often require repetitive conversion for handling timestamp formats, which is more cumbersome compared to q-sql, where time-based operations like xbar (interval-based bucketing) can be done directly in a streamlined manner. Temporal primitives also make it extremely easy to convert a nanosecond timestamp to it's equivalent minute using dot notation e.g. time.minute
- **Data Transformation**: The q language is optimized for high-performance, in-memory, columnar data transformations, which allows for more compact expressions on vectors of data. SQL, on the other hand, is typically too general purpose for even simple transformations on time-series data. This is down to how KDB-X/q is designed, where operations execute on ordered lists, whereas SQL (based on set theory) treats data as records instead of columns e.g. selecting the (last) value in a series, or understanding prior states (deltas) for series movements would require re-ordering the column data
- **Performance Considerations**: q-sql is designed for high-performance analytics on large datasets, and many operations that would require complex SQL expressions can be done efficiently with q-sql syntax. In SQL, complex operations requires workarounds such as additional processing with temporary tables, sub-expressions, re-indexing, changing data models, or heavily leveraging partitions and window functions.

Thus, while the core logic of the query is similar in both languages, the SQL version requires much more overhead in terms of complexity and verbosity. This inefficiency will also become more pronounced with large datasets, leading to challenges with query performance.

While these are just basic analytics, they highlight KDB-X/q’s ability to storage and analyse large-scale time-series datasets quickly.

## 6. Asof Join – Matching Trades with Quotes

One of the most powerful features in KDB-X/q is the asof join (`aj`), this is accessible via Python as the `merge_asof` method on table objects and is designed to match records from two tables based on the most recent timestamp. Unlike a standard SQL join, where records must match exactly on a key, an asof join finds the most recent match.

Why Use Asof Joins?

In time-series data, we often deal with information arriving at different intervals. For example:

- Trade and Quote Data: A trade occurs at a given time, and we want to match it with the latest available quote.
- Sensor Data: A sensor records temperature every second, while another logs environmental data every 10 seconds—matching the closest reading is crucial.


#### Generate synthetic quote data for one day

In [18]:
quote = kx.Table(data = {
     'time': day + kx.random.random(n, kx.q('24:00:00')),
     'sym': kx.random.random(n, syms),
     'bid': kx.random.random(n, 100.0),
     'ask': kx.random.random(n, 1000)
    }
).sort_values('time')

As we're keeping this table in memory we need to perform one extra step before joining, we apply the parted attribute to the sym column of the quote table. Our trade table on disk already has the parted attribute on the sym column, we see this in the column `a` when we run the following:

In [ ]:
database.trade.dtypes

,t,f,a
c,,,
date,"""d""",,
sym,"""s""",,p
time,"""p""",,
price,"""f""",,
size,"""j""",,


This is crucial for optimizing asof joins, as it ensures faster lookups when performing symbol-based joins. Before applying parted to quote, we first sort the table by sym using [`kx.q.xasc`](#https://code.kx.com/pykx/3.1/api/pykx-execution/q.html#xasc), as the parted attribute requires the column to be sorted for it to work efficiently.

In [ ]:
quote = quote.sort_values('sym')
quote = quote.parted('sym')

In the above:
- `quote.sort_values('sym')` Sorts the quote table by sym in ascending order
- `parted`  Applies the parted attribute to sym, optimizing symbol-based lookups.

#### Peform Asof Join

We now match each trade with the most recent available quote for todays date using <a href="https://code.kx.com/pykx/3.1/user-guide/advanced/Pandas_API.html#tablemerge_asof" target="_blank">merge_asof</a>.

In [21]:
subtrade = database.trade.select(where=kx.Column('date') == day)
subtrade.merge_asof(quote, ['sym', 'time'])

,date,sym,time,price,size,bid,ask
,,,,,,,
0,2025.01.01,ACOG,2025.01.01D00:00:00.000000000,90.01866,930,74.05334,508
1,2025.01.01,ACOG,2025.01.01D00:00:00.000000000,2.025345,688,74.05334,508
2,2025.01.01,ACOG,2025.01.01D00:00:01.000000000,24.0999,429,42.02724,528
3,2025.01.01,ACOG,2025.01.01D00:00:02.000000000,5.307969,706,18.45691,765
4,2025.01.01,ACOG,2025.01.01D00:00:02.000000000,38.31707,330,18.45691,765
5,2025.01.01,ACOG,2025.01.01D00:00:03.000000000,93.01008,74,64.68624,883
6,2025.01.01,ACOG,2025.01.01D00:00:03.000000000,77.77348,454,64.68624,883
7,2025.01.01,ACOG,2025.01.01D00:00:04.000000000,78.41395,130,8.597881,714
8,2025.01.01,ACOG,2025.01.01D00:00:05.000000000,5.631432,406,8.597881,714


In the above:
- `merge_asof` performs an asof join on the `sym` and `time` columns
- Each trade record gets matched with the latest available quote at or before the trade’s timestamp.

This approach ensures that for every trade, we have the best available quote information, allowing traders to analyze trade execution relative to the prevailing bid/ask spread at the time.